Assuming we have a dataframe with corpus/frequency data (Futrell et al., 2021) combined with GPT-2 surprisal and cosine similarities

In [ ]:
library(lme4)
library(data.table)
library(dplyr)
library(ggplot2)
library(broom.mixed)
library(tibble)


Load Surprisal Data

In [ ]:
surprisal_data = read_csv("gpt2_NS.csv") %>%
  separate(label, into = c("label", "label_num"), sep = "/") %>%
  mutate(label_num = str_trim(str_replace(label_num, pattern = ".word", replacement = "")))

story_info_surp = story_info %>%
  left_join(surprisal_data, by=c("word_num" = "label_num"))

Function to extract ERP data from raw EEG

In [ ]:
extract_ERP = function(subject, word_num, all_data, story_info,
                       ERP_window_start = -750, ERP_window_end = 1500,
                       channels ){
  cat(subject, "\n", word_num)
  word_onset = story_info$Onset[story_info$word_num == word_num]*1000
  word_onset_bin = round(word_onset/10)*10
  #cat(word_onset)
  story = story_info$Filename[story_info$word_num == word_num]
  #cat(story)

  ERP_data = all_data %>%
    filter(StoryFilename == story & Subject == subject) %>%
    filter(between(timebin_rel_story_onset, word_onset_bin + ERP_window_start, word_onset_bin+ERP_window_end))

  ERP_baseline_data = ERP_data %>%
    filter(between(timebin_rel_story_onset, word_onset_bin + ERP_window_start, word_onset_bin)) %>%
    dplyr::select(all_of(channels))

  ERP_baselines = colMeans(ERP_baseline_data)
  #cat(ERP_baselines["Fp1"][1])

  #ERP_window_end_rd = if_else(length(ERP_data$timebin_rel_story_onset)==120, ERP_window_end-10, ERP_window_end)

  ERP_data_baselined = ERP_data %>%
    dplyr::select(all_of(channels), StoryFilename, Subject, timebin_rel_story_onset) %>%
    mutate(word_num = word_num,
           word_onset_bin = word_onset_bin,
           across(all_of(channels), ~ .x - ERP_baselines[cur_column()][1], .names = "{.col}_b"),
           timebin = seq(from=ERP_window_start, to = ERP_window_end, by=10))

  return(ERP_data_baselined)
}

Align stories and subjects

In [ ]:
story_info_surp$word_num %>% unique() %>% length()
# 10310

good_word_nums = story_info_surp %>% filter(!is.na(label)) %>% pull(word_num)
good_word_nums %>% length()
# 5203

subjects <- unique(all_data$Subject)

erps = expand_grid(subjects, good_word_nums)
erps <- erps %>% mutate(story = sub("\\..*", "", good_word_nums))
erps$story <- as.numeric(erps$story)
erps$subjects %>% length()

# Create a new dataframe with unique subjects and their associated story filenames
unique_subjects_df <- all_data %>%
  group_by(Subject) %>%
  summarise(Story_Filenames = list(unique(StoryFilename))) %>%
  ungroup()

story_lookup <- unique_subjects_df %>%
  select(Subject, Story_Filenames) %>%
  deframe()

# Function to filter based on story lookup
filter_stories <- function(subject, story) {
  if (subject %in% names(story_lookup)) {
    return(story %in% story_lookup[[subject]])
  } else {
    return(FALSE)
  }
}

# Filter df2 based on the lookup list
erps <- erps %>%
  rowwise() %>%
  mutate(valid_story = filter_stories(subjects, story)) %>%
  filter(valid_story) %>%
  select(-valid_story) %>%
  ungroup()

all_erp_data = map2_df(.x=erps$subjects, .y=erps$good_word_nums, .f=~extract_ERP(subject = .x, word_num=.y, all_data = all_data, story_info = story_info_surp, channels = channel_names$X2 ))
saveRDS(all_erp_data, "all_erp_data.rds")


Align surprisal and ERP data

In [ ]:
firstQ = summary(story_info_surp$LSTM_batchnhid)[[2]]
thirdQ = summary(story_info_surp$LSTM_batchnhid)[[5]]

all_erp_data_surps = erp_data %>%
  left_join(story_info_surp, by = "word_num") %>%
  #select(-all_of(channel_names$X2)) %>%
  mutate(LSTM_surprisal_cat = case_when(
    LSTM_batchnhid <= firstQ ~ "low",
    LSTM_batchnhid > firstQ & RNN < thirdQ ~ "mid",
    LSTM_batchnhid >= thirdQ ~ "high"))

all_erp_data_surps <- all_erp_data_surps %>%
  mutate(LSTM_surprisal_cat = case_when(
    LSTM_batchnhid <= firstQ ~ "low",
    LSTM_batchnhid > firstQ & LSTM_batchnhid < thirdQ ~ "mid",
    LSTM_batchnhid >= thirdQ ~ "high"))

  all_erp_data_surps2  = all_erp_data_surps %>%
  #sample_n(20) %>%
  rowwise() %>%
  mutate(
    AvgN4channels = mean(c(Cz_b, C1_b, C2_b, C3_b, C4_b, CPz_b, CP1_b,
                           CP2_b, CP3_b, CP4_b, Pz_b, P1_b, P2_b, P3_b,
                           P4_b)))

Create ERP data for each time window in the epoch

In [ ]:
all_erp_data_surps2 <- as.data.table(readRDS("all_erp_data_surps2_complete.rds"))

N4_data_surps_200_0 = all_erp_data_surps2 %>%
  filter(timebin >= -200 & timebin <= 0) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_200_0$prev_freq <- lead(N4_data_surps_200_0$frequency, n = 1)
N4_data_surps_200_0$prev_word_cos_sim <- lead(N4_data_surps_200_0$word_cos_sim, n = 1)
N4_data_surps_200_0$prev_surp <- lead(N4_data_surps_200_0$gpt3_probs, n = 1)

N4_data_surps_0_200 = all_erp_data_surps2 %>%
  filter(timebin >= 0 & timebin <= 200) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_100_100 = all_erp_data_surps2 %>%                                                                                          filter(timebin >= -100 & timebin <= 100) %>%                                                                                           group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_100_100$prev_freq <- lead(N4_data_surps_100_100$frequency, n = 1)
N4_data_surps_100_100$prev_word_cos_sim <- lead(N4_data_surps_100_100$word_cos_sim, n = 1)
N4_data_surps_100_100$prev_surp <- lead(N4_data_surps_100_100$gpt3_probs, n = 1)

N4_data_surps_0_200$prev_freq <- lead(N4_data_surps_0_200$frequency, n = 1)
N4_data_surps_0_200$prev_word_cos_sim <- lead(N4_data_surps_0_200$word_cos_sim, n = 1)
N4_data_surps_0_200$prev_surp <- lead(N4_data_surps_0_200$gpt3_probs, n = 1)

N4_data_surps_100_300 = all_erp_data_surps2 %>%
  filter(timebin >= 100 & timebin <= 300) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_100_300$prev_freq <- lead(N4_data_surps_100_300$frequency, n = 1)
N4_data_surps_100_300$prev_word_cos_sim <- lead(N4_data_surps_100_300$word_cos_sim, n = 1)
N4_data_surps_100_300$prev_surp <- lead(N4_data_surps_100_300$gpt3_probs, n = 1)

N4_data_surps_200_400 = all_erp_data_surps2 %>%
  filter(timebin >= 200 & timebin <= 400) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_200_400$prev_freq <- lead(N4_data_surps_200_400$frequency, n = 1)
N4_data_surps_200_400$prev_word_cos_sim <- lead(N4_data_surps_200_400$word_cos_sim, n = 1)
N4_data_surps_200_400$prev_surp <- lead(N4_data_surps_200_400$gpt3_probs, n = 1)

N4_data_surps_300_500 = all_erp_data_surps2 %>%
  filter(timebin >= 300 & timebin <= 500) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_300_500$prev_freq <- lead(N4_data_surps_300_500$frequency, n = 1)
N4_data_surps_300_500$prev_word_cos_sim <- lead(N4_data_surps_300_500$word_cos_sim, n = 1)
N4_data_surps_300_500$prev_surp <- lead(N4_data_surps_300_500$gpt3_probs, n = 1)

N4_data_surps_400_600 = all_erp_data_surps2 %>%
  filter(timebin >= 400 & timebin <= 600) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_400_600$prev_freq <- lead(N4_data_surps_400_600$frequency, n = 1)
N4_data_surps_400_600$prev_word_cos_sim <- lead(N4_data_surps_400_600$word_cos_sim, n = 1)
N4_data_surps_400_600$prev_surp <- lead(N4_data_surps_400_600$gpt3_probs, n = 1)

N4_data_surps_500_700 = all_erp_data_surps2 %>%
  filter(timebin >= 500 & timebin <= 700) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_500_700$prev_freq <- lead(N4_data_surps_500_700$frequency, n = 1)
N4_data_surps_500_700$prev_word_cos_sim <- lead(N4_data_surps_500_700$word_cos_sim, n = 1)
N4_data_surps_500_700$prev_surp <- lead(N4_data_surps_500_700$gpt3_probs, n = 1)

N4_data_surps_600_800 = all_erp_data_surps2 %>%
  filter(timebin >= 600 & timebin <= 800) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_600_800$prev_freq <- lead(N4_data_surps_600_800$frequency, n = 1)
N4_data_surps_600_800$prev_word_cos_sim <- lead(N4_data_surps_600_800$word_cos_sim, n = 1)
N4_data_surps_600_800$prev_surp <- lead(N4_data_surps_600_800$gpt3_probs, n = 1)

N4_data_surps_700_900 = all_erp_data_surps2 %>%
  filter(timebin >= 700 & timebin <= 900) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_700_900$prev_freq <- lead(N4_data_surps_700_900$frequency, n = 1)
N4_data_surps_700_900$prev_word_cos_sim <- lead(N4_data_surps_700_900$word_cos_sim, n = 1)
N4_data_surps_700_900$prev_surp <- lead(N4_data_surps_700_900$gpt3_probs, n = 1)

N4_data_surps_800_1000 = all_erp_data_surps2 %>%
  filter(timebin >= 800 & timebin <= 1000) %>%
  group_by(Subject, word_num, StoryFilename, full_word, sentpos, RNN, LSTM_batchnhid, residuals_ltanh_batchnhid, LSTM_surprisal_cat, frequency, word_cos_sim, Duration, gpt3_probs, sentid) %>%
  summarize(meanN4 = mean(AvgN4channels)) %>%
  mutate(wlen = nchar(full_word))

N4_data_surps_800_1000$prev_freq <- lead(N4_data_surps_800_1000$frequency, n = 1)
N4_data_surps_800_1000$prev_word_cos_sim <- lead(N4_data_surps_800_1000$word_cos_sim, n = 1)
N4_data_surps_800_1000$prev_surp <- lead(N4_data_surps_800_1000$gpt3_probs, n = 1)


#vanilla_lmer <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_300_500)

#saveRDS(vanilla_lmer, "vanilla_fullresidlm4.rds")

#myvar <- allFit(vanilla_lmer)

#saveRDS(myvar, "allfitresid_vanilla.rds")

#lm1 <- lmer(meanN4 ~ scale(residuals_ltanh_batchnhid) + scale(log(frequency)) + scale(log(prev_freq)) + scale(word_cos_sim) + scale(prev_word_cos_sim) + scale(gpt3_probs) + scale(prev_surp) + scale(Duration) + scale(sentpos) + scale(sentid) + (1|full_word) + (1|StoryFilename) + (1 + scale(residuals_ltanh_batchnhid) + scale(log(frequency)) + scale(log(prev_freq)) + scale(word_cos_sim) + scale(prev_word_cos_sim) + scale(gpt3_probs) + scale(prev_surp) + scale(Duration) + scale(sentpos) + scale(sentid) | Subject), data = N4_data_surps_0_200, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

#saveRDS(lm1, "scalefullresidlm1.rds")

lm1 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_0_200, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm1, "fulloptresidlm1.rds")

#lm2 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_100_300)

#saveRDS(lm2, "fullresidlm2.rds")

lm2 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_100_300, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm2, "fulloptresidlm2.rds")

#lm3 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_200_400)

#saveRDS(lm3, "fullresidlm3.rds")

lm3 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_200_400, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm3, "fulloptresidlm3.rds")

#lm4 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid || Subject), data = N4_data_surps_300_500)

#saveRDS(lm4, "fullresidlm4_nofullword.rds")

#myvar <- allFit(lm4)

#saveRDS(myvar, "allfitresid_n400output.rds")

lm4 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_300_500, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm4, "fulloptresidlm4.rds")

#lm5 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_400_600)

#saveRDS(lm5, "fullresidlm5.rds")

lm5 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_400_600, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm5, "fulloptresidlm5.rds")

#lm6 <- lmer(meanN4 ~ scale(residuals_ltanh_batchnhid) + scale(log(frequency)) + scale(log(prev_freq)) + scale(word_cos_sim) + scale(prev_word_cos_sim) + scale(gpt3_probs) + scale(prev_surp) + scale(Duration) + scale(sentpos) + (1|full_word) + (1|StoryFilename) + (1 + scale(residuals_ltanh_batchnhid) + scale(log(frequency)) + scale(log(prev_freq)) + scale(word_cos_sim) + scale(prev_word_cos_sim) + scale(gpt3_probs) + scale(prev_surp) + scale(Duration) + scale(sentpos) || Subject), data = N4_data_surps_500_700)

#lm6 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_500_700, control = lmerControl(optimizer = "bobyqa", , optCtrl = list(maxfun = 5e+05)))

#saveRDS(lm6, "fullresidlm6-optbobyqa.rds")

#my_var <- allFit(lm6)

#saveRDS(my_var, "allfitresid_output6.rds")

lm6 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_500_700, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm6, "fulloptresidlm6.rds")

#lm7 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_600_800)

#saveRDS(lm7, "fullresidlm7.rds")

lm7 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_600_800, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm7, "fulloptresidlm7.rds")

#lm8 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_700_900)

#saveRDS(lm8, "fullresidlm8.rds")

lm8 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_700_900, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm8, "fulloptresidlm8.rds")

#lm9 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|full_word) + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid | Subject), data = N4_data_surps_800_1000)

#saveRDS(lm9, "fullresidlm9.rds")

lm9 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_800_1000, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm9, "fulloptresidlm9.rds")

lm10 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_100_100, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm10, "fulloptresidlm10.rds")

lm11 <- lmer(meanN4 ~ residuals_ltanh_batchnhid + scale(log(frequency)) + scale(log(prev_freq)) + word_cos_sim + prev_word_cos_sim + gpt3_probs + prev_surp + Duration + sentpos + sentid + (1|StoryFilename) + (1 + residuals_ltanh_batchnhid + word_cos_sim + gpt3_probs || Subject), data = N4_data_surps_200_0, control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=2e5)))

saveRDS(lm11, "fulloptresidlm11.rds")


Set model and time window variables

In [ ]:
models <- list(
  lm11,  # -200–0
  lm10,  # -100–100
  lm1,   # 0–200
  lm2,   # 100–300
  lm3,   # 200–400
  lm4,   # 300–500
  lm5,   # 400–600
  lm6,   # 500–700
  lm7,   # 600–800
  lm8,   # 700–900
  lm9    # 800–1000
)

time_windows <- tibble(
  window_id  = seq_along(models),
  start_ms = c(-200, -100,   0, 100, 200, 300, 400, 500, 600, 700, 800),
  end_ms   = c(   0,  100, 200, 300, 400, 500, 600, 700, 800, 900, 1000)
) %>%
  mutate(time_center = (start_ms + end_ms) / 2)



Extract fixed effects

In [ ]:
predictors_of_interest <- c(
  "gpt3_probs",
  "residuals_ltanh_batchnhid",
  "word_cos_sim"
)

extract_model <- function(model, window_id) {
  broom.mixed::tidy(model, effects = "fixed") %>%
    filter(term %in% predictors_of_interest) %>%
    mutate(window_id = window_id)
}

erp_results <- bind_rows(
  lapply(seq_along(models), function(i) {
    extract_model(models[[i]], i)
  })
)
erp_results <- erp_results %>%
  left_join(time_windows, by = "window_id") %>%
  mutate(
    ci_low  = estimate - 1.96 * std.error,
    ci_high = estimate + 1.96 * std.error
  )



Add time + confidence intervals, label predictors for facets

In [ ]:
erp_results <- erp_results %>%
  mutate(
    term_label = recode(
      term,
      "gpt3_probs"                = "Surprisal β",
      "residuals_ltanh_batchnhid" = "Residual β",
      "word_cos_sim"              = "Cosine Similarity β"
    )
  )

erp_results$term_label <- factor(
  erp_results$term_label,
  levels = c("Surprisal β", "Residual β", "Cosine Similarity β")
)


Plot results to see where in the timecourse residual, surprisal, and cosine similarity are significant

In [ ]:
ggplot(
  erp_results,
  aes(
    x = time_center,
    y = estimate,
    color = term_label,
    fill  = term_label
  )
) +
  geom_ribbon(
    aes(ymin = ci_low, ymax = ci_high),
    alpha = 0.25,
    linewidth = 0
  ) +
  geom_line(linewidth = 1) +
  geom_hline(yintercept = 0, linewidth = 0.4) +
  geom_vline(xintercept = 0, linewidth = 0.4) +
  facet_wrap(~ term_label, ncol = 1, scales = "free_y") +
  scale_color_manual(values = c(
    "Surprisal β"         = "#2C6BED",
    "Residual β"         = "#C23B22",
    "Cosine Similarity β" = "#8E44AD"
  )) +
  scale_fill_manual(values = c(
    "Surprisal β"         = "#2C6BED",
    "Residual β"         = "#C23B22",
    "Cosine Similarity β" = "#8E44AD"
  )) +
  labs(
    x = "Time Window (ms)",
    y = expression(beta * "-value")
  ) +
  theme_minimal(base_size = 12) +
  theme(
    strip.text = element_text(face = "bold"),
    panel.grid.minor = element_blank(),
    legend.position = "none"
  )
